Using these two as references: 
https://gist.github.com/Smerity/afe7430fdb4371015466
https://gist.github.com/Smerity/56bc6f21a8adec920ebf

Python 3 following code written in Python 2.7

In [17]:
import gzip
import json
import requests
import glob
import pandas as pd
from io import StringIO, BytesIO

In [3]:
urls = pd.read_csv("corpus_urls.csv")
start_url = urls.iloc[1,0]

In [4]:
start_url

'www.anoki.co.uk'

Will start downloading the WARC files. 
Then cycle through them and process them

In [5]:
pg_prefix = 'https://commoncrawl.s3.amazonaws.com/'

In [6]:
index_prefix = 'http://index.commoncrawl.org/CC-MAIN-2018-09-index?url='

In [7]:
page_resp = requests.get(index_prefix+start_url+'/*&output=json&limit=1')

In [8]:
page_resp.status_code == 200

True

In [9]:
pg_nm = page_resp.json()['filename']

In [10]:
pg_nm

'crawl-data/CC-MAIN-2018-09/segments/1518891812978.31/crawldiagnostics/CC-MAIN-20180220145713-20180220165713-00150.warc.gz'

In [11]:
offset, length = int(page_resp.json()['offset']), int(page_resp.json()['length'])
offset_end = offset + length -1

In [12]:
pg_resp = requests.get(pg_prefix+pg_nm, headers={'Range':'bytes={}-{}'.format(offset,offset_end)})

In [13]:
rawdata = BytesIO(pg_resp.content)

In [14]:
file = gzip.GzipFile(fileobj=rawdata)

In [15]:
data = file.read()

In [25]:
warc, header, response = data.strip().split(b'\r\n\r\n', 2)